In [ ]:
import os
import numpy as np
import torch

#import torchvision.transforms as transforms
from torchvision import datasets
from torchvision import transforms

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path = '/scratch/ssd/cciw/dataset_voc/'
file = '1349_2016-07-06_2_GLN_3061_crop'

label_path = os.path.join(os.path.join(path, 'SegmentationClassPNG'), file)
image_path = os.path.join(os.path.join(path, 'JPEGImages'), file)

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
# for debugging purposes
# dlpath = '/scratch/gallowaa/'
# trainset = ds.VOCSegmentation(dlpath, image_set='val', download=True, transform=transform)
trainset = ds.VOCSegmentation(
    root='/scratch/ssd/cciw/', year='2012', image_set='train',
    download=False, transform=transform, target_transform=transform
)

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=5, shuffle=True)

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
torch.cuda.device_count()

In [ ]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

In [ ]:
for inputs, targets in trainloader:
    inputs, targets = inputs.cuda(), targets.cuda()
print(inputs.shape)
print(targets.shape)

In [ ]:
idx = 0
plt.imshow(inputs.detach().cpu().numpy()[idx, 0])
#plt.imshow(targets.detach().cpu().numpy()[idx, 0])

In [ ]:
np.unique(targets.detach().cpu().numpy()[1, 0])

In [ ]:
%matplotlib inline

In [ ]:
#1/255

In [ ]:
from torchvision.models.segmentation import fcn_resnet50

In [ ]:
net = fcn_resnet50(num_classes=2).cuda()

In [ ]:
net.eval()
print(net.training)

In [ ]:
pred = net(inputs)['out']

In [ ]:
plt.imshow(pred.detach().cpu().numpy()[idx, 0])

In [ ]:
plt.imshow(pred.detach().cpu().numpy()[idx, 1])

In [ ]:
plt.imshow(torch.argmax(pred, dim=1).detach().cpu().numpy()[idx])

In [ ]:
pred.shape

In [ ]:
import torch.nn as nn

In [ ]:
#pixel_acc(targets, targets)
pixel_acc(pred, pred)

In [ ]:
targets.shape

In [ ]:
def pixel_acc(pred, label):
    #_, preds = torch.max(pred, dim=1)
    preds = torch.argmax(pred, dim=1)
    valid = (label >= 0).long()
    acc_sum = torch.sum(valid * (preds == label).long())
    pixel_sum = torch.sum(valid)
    acc = acc_sum.float() / (pixel_sum.float() + 1e-10)
    return acc

In [ ]:
class SegmentationModuleBase(nn.Module):
    def __init__(self):
        super(SegmentationModuleBase, self).__init__()

    def pixel_acc(self, pred, label):
        _, preds = torch.max(pred, dim=1)
        valid = (label >= 0).long()
        acc_sum = torch.sum(valid * (preds == label).long())
        pixel_sum = torch.sum(valid)
        acc = acc_sum.float() / (pixel_sum.float() + 1e-10)
        return acc


class SegmentationModule(SegmentationModuleBase):
    def __init__(self, net_enc, net_dec, crit, deep_sup_scale=None):
        super(SegmentationModule, self).__init__()
        self.encoder = net_enc
        self.decoder = net_dec
        self.crit = crit
        self.deep_sup_scale = deep_sup_scale

    def forward(self, feed_dict, *, segSize=None):
        # training
        if segSize is None:
            if self.deep_sup_scale is not None: # use deep supervision technique
                (pred, pred_deepsup) = self.decoder(self.encoder(feed_dict['img_data'], return_feature_maps=True))
            else:
                pred = self.decoder(self.encoder(feed_dict['img_data'], return_feature_maps=True))

            loss = self.crit(pred, feed_dict['seg_label'])
            if self.deep_sup_scale is not None:
                loss_deepsup = self.crit(pred_deepsup, feed_dict['seg_label'])
                loss = loss + loss_deepsup * self.deep_sup_scale

            acc = self.pixel_acc(pred, feed_dict['seg_label'])
            return loss, acc
        # inference
        else:
            pred = self.decoder(self.encoder(feed_dict['img_data'], return_feature_maps=True), segSize=segSize)
            return pred